In [ ]:
# Load data - run prep_exp.ipynb first
import pickle

pickle_in = open("temp.pkl", "rb")
data = pickle.load(pickle_in)

# # If it's not there, do the following:
# import sys
# import os
# import numpy as np
# from str2bool import str2bool
# from rpy2.robjects.packages import STAP
# import rpy2.robjects as robjects
# from rpy2.robjects import pandas2ri, numpy2ri
# from rpy2.robjects.lib.dplyr import DataFrame
# from rpy2.robjects.packages import importr

# pandas2ri.activate()
# numpy2ri.activate()

# # Some temporary arguments for testing
# include_lab = str2bool("T")  # Include lab features?
# include_ethdon = str2bool("T")  # Include ethnicity + donor details?
# lag = int("1")  # Number of lag variables
# eq_train_ratio = str2bool("T")  # Train on equal case:control ratio?
# __file__ = '/h/angeliney/projects/SRTR/prep_exp.py'
# visit_type = "first"
# output = "temp"
# post2000 = True

# # Get features based on these inputs
# with open(os.path.join(os.path.dirname(__file__), 'features.R'), 'r') as f:
#     string = f.read()
# features_file = STAP(string, "features")
# features_to_use = features_file.features.rx2("clin")
# if include_lab:
#     features_to_use = features_to_use + features_file.features.rx2("lab")

# if include_ethdon:
#     features_to_use = features_to_use + features_file.features.rx2("eth") + features_file.features.rx2("don")

# timedep_cols = np.intersect1d(features_to_use, features_file.timedep_features)
# cov_cols = np.setdiff1d(features_to_use, timedep_cols)

# if eq_train_ratio:
#     eq_cases_train_cols = np.array(["TRR_ID", "is_diab"])
# else:
#     eq_cases_train_cols = np.array()
    
# # Read RDS files (load data table)
# readRDS = robjects.r['readRDS']
# tx_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'tx_li_formatted.rds'))
# txf_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'txf_li_formatted.rds'))

# # Merge them
# with open(os.path.join(os.path.dirname(__file__), 'functions.R'), 'r') as f:
#     string = f.read()
# functions = STAP(string, "functions")

# merged = functions.combine_tx_txf(tx_li_study, txf_li_study, np.setdiff1d(cov_cols, "age"), timedep_cols, lag)

# df = pandas2ri.ri2py_dataframe(DataFrame(merged).filter('time_next_followup > time_since_transplant'))

# #Prep data for model training
# cols = np.concatenate((timedep_cols, cov_cols))
# if lag > 0:
#     for l in range(1,  lag + 1):
#         cols = np.append(cols, list(map(lambda x: '{}_{}'.format(x, l), timedep_cols)))

# subset_cols = np.concatenate((['transplant_year', 'TRR_ID', 'age'], cols, ['is_diab', 'time_since_transplant',
#                                                                    'time_next_followup', 'time_to_diab',
#                                                                    'diab_time_since_tx', 'diab_in_1_year',
#                                                                    'diab_now']))
# df = df.dropna(subset=subset_cols)
# df_test = df[(df.transplant_year.astype(int) >= 2011) & (df.time_to_diab >= 0)]
# df_nontest = df[(df.transplant_year.astype(int) < 2011) & (df.time_to_diab >= 0)]

# num_folds = 5
# nontest_y = df_nontest.drop_duplicates(subset=['TRR_ID', 'is_diab']).is_diab
# caret = importr('caret')
# folds = caret.createFolds(nontest_y, num_folds, False)

# data = {'test': df_test, 'train': df_nontest, 'cols': cols, 'eq_cases_train_cols': eq_cases_train_cols,
#             'folds': folds}

# import pickle

# # Save data in case kernel got restarted
# pickle.dump(data, open("temp.pkl", "wb")) 

In [ ]:
# Try DeepSurv
import numpy as np
import pandas
import sys
import lasagne
import deepsurv
import optunity
from str2bool import str2bool
from functions import prep_exp, filter_train_by_visit


nontest_ids = data['train'].drop_duplicates(subset=['TRR_ID', 'is_diab']).TRR_ID
visit_type="random"
if visit_type == "last":
    time_col = "time_since_transplant"
    cols = data['cols']
else:
    time_col = "time_to_diab"
    cols = np.append(data['cols'], 'time_since_transplant')

    
train = filter_train_by_visit("random", data['train'])

In [ ]:
# Find best hyperparameter setting
def get_objective_function(num_epochs, update_fn=lasagne.updates.adam):
    """
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to
    use as hyper-parameters.
    The objective function runs a DeepSurv model on the training data and evaluates it against the
    test set for validation. The result of the function call is the validation concordance index
    (which Optunity tries to optimize)
    """
    def format_to_deepsurv(x, y):
        return {
            'x': x.astype(np.float32),
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }
    
    def get_hyperparams(params):
        hyperparams = {
            'batch_norm': True,
            'standardize': True
        }
        if 'num_layers' in params and 'num_nodes' in params:
            params['hidden_layers_sizes'] = [int(params['num_nodes'])] * int(params['num_layers'])
            del params['num_layers']
            del params['num_nodes']
        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
        hyperparams.update(params)
        return hyperparams
    def train_deepsurv(x_train, y_train, x_test, y_test, **kwargs):
        # Standardize the datasets
        train_mean = x_train.mean(axis = 0)
        train_std = x_train.std(axis = 0)
        x_train = (x_train - train_mean) / train_std
        x_test = (x_test - train_mean) / train_std

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)
        
        hyperparams = get_hyperparams(kwargs)

        network = deepsurv.DeepSurv(n_in=train_data['x'].shape[1], **hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs,
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        return result

    return train_deepsurv

update_fn = lasagne.updates.momentum
opt_fxn = get_objective_function(100, update_fn=update_fn)



box_constraints = {
    "learning_rate": [-7, -3],
    "num_nodes": [2, 20],
    "num_layers": [1, 4],
    "lr_decay": [0.0, 0.001],
    "momentum": [0.8, 0.95],
    "L1_reg": [0.05, 5.0],
    "L2_reg": [0.05, 5.0],
    "dropout": [0.0, 0.5]
}

opt_fxn = optunity.cross_validated(x=train[cols].values,
                                   y=np.column_stack((train.is_diab.values, train[time_col].values)),
                                   num_folds=5)(opt_fxn)
opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=100, solver_name='sobol', **box_constraints)
hyperparams = opt_params

In [ ]:
result = pandas.DataFrame()
i = 0
if i > 0:
    train_ids = nontest_ids[np.array(data['folds']) != i]
    val_ids = np.setdiff1d(nontest_ids, train_ids)
    test = data['train'][data['train']['TRR_ID'].isin(val_ids)]
else:
    train_ids = nontest_ids
    test = data['test']

train = filter_train_by_visit(visit_type, data['train'][data['train'].TRR_ID.isin(train_ids)])

train_data = {"x": train[cols].values.astype("float32"),
              "t": train[time_col].values.astype("float32"),
              "e": train.is_diab.values.astype("int32")}

test_data = {"x": test[cols].values.astype("float32"),
             "t": test[time_col].values.astype("float32"),
             "e": test.is_diab.values.astype("int32")}

network = deepsurv.DeepSurv(**hyperparams)
log = network.train(**train_data, n_epochs=100, update_fn=update_fn)
train_cindex = network.get_concordance_index(**train_data)
test_cindex = network.get_concordance_index(**test_data)

# Get c-index case
train_case_data = {"x": train.query('is_diab == 1')[cols].values.astype("float32"),
                   "t": train.query('is_diab == 1')[time_col].values.astype("float32"),
                   "e": train.query('is_diab == 1').is_diab.values.astype("int32")}
test_case_data = {"x": test.query('is_diab == 1')[cols].values.astype("float32"),
                  "t": test.query('is_diab == 1')[time_col].values.astype("float32"),
                  "e": test.query('is_diab == 1').is_diab.values.astype("int32")}
train_cindex_case = network.get_concordance_index(**train_case_data)
test_cindex_case = network.get_concordance_index(**test_case_data)

if i == 0:
    perf = {'model': network, 'train_cindex': train_cindex, 'test_cindex': test_cindex,
            'train_nrow': train.shape[0], 'test_nrow': test.shape[0],
            'train_cindex_case': train_cindex_case, 'test_cindex_case': test_cindex_case}

print('{},{},{},{},{},{},{}'.format(i, train_cindex, test_cindex, train.shape[0], test.shape[0], train_cindex_case,
                                    test_cindex_case))
result.append(pandas.DataFrame({'train_cindex': train_cindex, 'test_cindex': test_cindex,
                                'train_nrow': train.shape[0], 'test_nrow': test.shape[0],
                                'train_cindex_case': train_cindex_case, 'test_cindex_case': test_cindex_case},
                               index=[0]))